In [1]:
!nvidia-smi

Mon Mar 22 01:21:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   44C    P0    51W / 300W |   7912MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# setup

In [2]:
%load_ext autoreload
%autoreload 2
%aimport

HOME = '/data/git/shopee-product-matching'
import sys
sys.path.append(f"{HOME}/src")

p_in = f'{HOME}/input'
p_comp = f'{p_in}/shopee-product-matching'
p_out = f'{HOME}/output/dev0017'

!mkdir -p {p_out}
from utils import *
import tmdl
import vmdl
from vmdl import VDataset, tfms



Modules to reload:
all-except-skipped

Modules to skip:



In [3]:
p_imgs = f"{p_comp}/test_images"
df = pd.read_csv(f'{p_comp}/test.csv')
df.shape

(3, 4)

In [4]:
p_imgs = f"{p_comp}/train_images"
df = pd.read_csv(f'{p_comp}/train.csv', nrows=1000)
df.shape

(1000, 5)

In [5]:
pids = df.posting_id.values

In [6]:
from efficientnet_pytorch import EfficientNet
mdl = EfficientNet.from_pretrained("efficientnet-b0")

feats_v = vmdl.mk_feats(df, p_imgs, mdl)

Loaded pretrained weights for efficientnet-b0


In [7]:
from sklearn.neighbors import NearestNeighbors

In [8]:
%%time
nbrs = NearestNeighbors(n_neighbors=10, algorithm='auto').fit(feats_v)
distances, indices = nbrs.kneighbors(feats_v)

CPU times: user 53.6 ms, sys: 8.68 ms, total: 62.3 ms
Wall time: 61.6 ms


In [9]:
distances

array([[ 0.        , 11.07120937, 11.90917056, ..., 12.35675628,
        12.36066446, 12.41143023],
       [ 0.        ,  5.73584561,  5.94093856, ..., 12.47508   ,
        12.62790569, 12.84036322],
       [ 0.        , 13.31632692, 13.93538748, ..., 14.46509171,
        14.46917946, 14.48850865],
       ...,
       [ 0.        ,  0.70688084, 13.60541516, ..., 14.06157357,
        14.10978708, 14.13784247],
       [ 0.        , 12.89515191, 12.97666656, ..., 13.18804232,
        13.2386719 , 13.30479171],
       [ 0.        , 11.1464998 , 11.26975537, ..., 11.90071904,
        11.93154442, 12.05501483]])

In [10]:
indices

array([[  0, 686, 583, ..., 637, 968, 546],
       [  1, 469, 194, ..., 511, 553, 970],
       [  2, 435, 390, ..., 622, 140,  69],
       ...,
       [997, 520, 558, ..., 357, 185, 923],
       [998, 439, 234, ..., 830, 197,  96],
       [999, 288, 642, ..., 214, 430, 293]])